# The Arc of a Junior Baller
## Tracking Player Development in Victorian Basketball

Every parent on the sideline wonders the same thing: *is my kid getting better?*

With multi-season data from the Eastern Districts Junior Basketball Association (EDJBA), we can finally answer that question with data. This notebook tracks how junior basketballers develop over time — who improves, who plateaus, and what separates breakout players from the pack.

We'll follow thousands of players across up to 6 seasons, build scoring trajectories, identify the most improved players, and even attempt to predict future performance from early-career stats.

**Data source:** PlayHQ statistics via the FullCourtVision scraper — ~26,000 players, ~144,000 stat lines, spanning 2020–2026.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

conn = sqlite3.connect('../data/playhq.db')

# Build master dataset: player stats joined with season/grade info
df = pd.read_sql_query("""
    SELECT 
        ps.player_id,
        p.first_name,
        p.last_name,
        ps.grade_id,
        g.name as grade_name,
        g.season_id,
        s.name as season_name,
        s.start_date,
        ps.games_played,
        ps.total_points,
        ps.one_point,
        ps.two_point,
        ps.three_point,
        ps.total_fouls,
        ps.team_name
    FROM player_stats ps
    JOIN players p ON ps.player_id = p.id
    JOIN grades g ON ps.grade_id = g.id
    JOIN seasons s ON g.season_id = s.id
    WHERE ps.games_played > 0
""", conn)

# Extract age group from grade name (e.g. "Boys U14 BB" -> "U14")
df['age_group'] = df['grade_name'].str.extract(r'(U\d+)')
df['gender'] = df['grade_name'].apply(lambda x: 'Boys' if x.startswith('Boys') else 'Girls' if x.startswith('Girls') else 'Mixed')
df['ppg'] = df['total_points'] / df['games_played']

# Create a season ordering
season_order_map = {
    'Summer 2020/21': 1, 'Autumn 2021': 2, 'Winter 2021': 3, 'Spring 2021': 4,
    'Summer 2021/22': 5, 'Autumn 2022': 6, 'Winter 2022': 7, 'Spring 2022': 8,
    'Summer 2022/23': 9, 'Autumn 2023': 10, 'Winter 2023': 11, 'Spring 2023': 12,
    'Summer 2023/24': 13, 'Autumn 2024': 14, 'Winter 2024': 15, 'Spring 2024': 16,
    'Summer 2024/25': 17, 'Autumn 2025': 18, 'Winter 2025': 19, 'Spring 2025': 20,
    'Summer 2025/26': 21, 'Autumn 2026': 22
}
# Also handle Term-based and year-only seasons
for sn in df['season_name'].unique():
    if sn not in season_order_map:
        # Try to extract year and assign approximate order
        import re
        m = re.search(r'(\d{4})', sn)
        if m:
            yr = int(m.group(1))
            season_order_map[sn] = (yr - 2020) * 4 + 2  # approximate

df['season_order'] = df['season_name'].map(season_order_map)

# Aggregate per player per season (a player might be in multiple grades in one season)
player_season = df.groupby(['player_id', 'first_name', 'last_name', 'season_name', 'season_order']).agg(
    games=('games_played', 'sum'),
    points=('total_points', 'sum'),
    fouls=('total_fouls', 'sum'),
    ft=('one_point', 'sum'),
    fg2=('two_point', 'sum'),
    fg3=('three_point', 'sum'),
    age_group=('age_group', 'first'),
    gender=('gender', 'first')
).reset_index()
player_season['ppg'] = player_season['points'] / player_season['games']
player_season = player_season.sort_values(['player_id', 'season_order'])

print(f"Total stat lines: {len(df):,}")
print(f"Unique players: {df['player_id'].nunique():,}")
print(f"Player-season combos: {len(player_season):,}")
print(f"Seasons covered: {df['season_name'].nunique()}")

Total stat lines: 150,304
Unique players: 26,319
Player-season combos: 70,029
Seasons covered: 8


## 🏀 Multi-Season Players: The Retention Funnel

Before we can study development, we need to know: how many kids actually stick around? Junior sport has notoriously high dropout rates. Let's see how our data reflects that.

In [2]:
# Count seasons per player
seasons_per_player = player_season.groupby('player_id').size().reset_index(name='num_seasons')

retention = seasons_per_player['num_seasons'].value_counts().sort_index()
cumulative = pd.DataFrame({
    'min_seasons': range(1, retention.index.max() + 1),
    'players': [seasons_per_player[seasons_per_player['num_seasons'] >= i].shape[0] for i in range(1, retention.index.max() + 1)]
})

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Players by Number of Seasons', 'Retention Funnel: Players with N+ Seasons'
))

fig.add_trace(go.Bar(
    x=retention.index, y=retention.values,
    marker_color='#FF6B35', text=retention.values, textposition='outside',
    name='Players'
), row=1, col=1)

fig.add_trace(go.Funnel(
    y=[f'{n}+ seasons' for n in cumulative['min_seasons']],
    x=cumulative['players'],
    textinfo='value+percent initial',
    marker_color=['#2196F3', '#4CAF50', '#FF9800', '#F44336', '#9C27B0', '#00BCD4'][:len(cumulative)]
), row=1, col=2)

fig.update_layout(
    title_text='Player Retention Across Seasons',
    height=450, showlegend=False,
    template='plotly_white'
)
fig.show()

total = len(seasons_per_player)
multi = seasons_per_player[seasons_per_player['num_seasons'] >= 2].shape[0]
three_plus = seasons_per_player[seasons_per_player['num_seasons'] >= 3].shape[0]
print(f"\n📊 {total:,} total players | {multi:,} ({multi/total:.0%}) played 2+ seasons | {three_plus:,} ({three_plus/total:.0%}) played 3+ seasons")


📊 26,319 total players | 15,652 (59%) played 2+ seasons | 12,193 (46%) played 3+ seasons


## 📈 Scoring Trajectories: Getting Better or Hitting a Wall?

For players with 3+ seasons of data, let's track their points-per-game over time. We'll normalize each player's timeline to "Season 1, Season 2, ..." so we can compare apples to apples.

In [3]:
# Players with 3+ seasons
multi_season_ids = seasons_per_player[seasons_per_player['num_seasons'] >= 3]['player_id']
multi_df = player_season[player_season['player_id'].isin(multi_season_ids)].copy()

# Assign sequential season number per player
multi_df['season_num'] = multi_df.groupby('player_id').cumcount() + 1

# Calculate trajectory direction: compare last season to first
trajectories = multi_df.groupby('player_id').agg(
    first_ppg=('ppg', 'first'),
    last_ppg=('ppg', 'last'),
    num_seasons=('season_num', 'max'),
    name=('first_name', 'first'),
    last_name=('last_name', 'first')
).reset_index()
trajectories['ppg_change'] = trajectories['last_ppg'] - trajectories['first_ppg']
trajectories['direction'] = pd.cut(trajectories['ppg_change'], bins=[-100, -1, 1, 100], labels=['Declining', 'Stable', 'Improving'])

# Summary
direction_counts = trajectories['direction'].value_counts()
fig = px.pie(
    values=direction_counts.values, names=direction_counts.index,
    color=direction_counts.index,
    color_discrete_map={'Improving': '#4CAF50', 'Stable': '#FF9800', 'Declining': '#F44336'},
    title='Scoring Trajectory Direction (3+ Season Players)',
    hole=0.4
)
fig.update_layout(height=400, template='plotly_white')
fig.show()

print(f"Improving (PPG up by 1+): {direction_counts.get('Improving', 0):,}")
print(f"Stable (within ±1 PPG):   {direction_counts.get('Stable', 0):,}")
print(f"Declining (PPG down 1+):  {direction_counts.get('Declining', 0):,}")

Improving (PPG up by 1+): 4,928
Stable (within ±1 PPG):   4,315
Declining (PPG down 1+):  2,950


In [4]:
# Spaghetti plot of scoring trajectories (sample for readability)
np.random.seed(42)
sample_ids = trajectories.sample(min(200, len(trajectories)))['player_id']
sample_df = multi_df[multi_df['player_id'].isin(sample_ids)]

fig = go.Figure()
for pid in sample_ids:
    pdata = sample_df[sample_df['player_id'] == pid]
    traj = trajectories[trajectories['player_id'] == pid].iloc[0]
    color = '#4CAF50' if traj['ppg_change'] > 1 else '#F44336' if traj['ppg_change'] < -1 else '#BDBDBD'
    fig.add_trace(go.Scatter(
        x=pdata['season_num'], y=pdata['ppg'],
        mode='lines', line=dict(width=0.8, color=color),
        opacity=0.4, showlegend=False,
        hovertext=f"{traj['name']} {traj['last_name']}"
    ))

# Add average line
avg_by_season = multi_df.groupby('season_num')['ppg'].mean()
fig.add_trace(go.Scatter(
    x=avg_by_season.index, y=avg_by_season.values,
    mode='lines+markers', line=dict(width=4, color='black'),
    name='Average PPG', marker=dict(size=8)
))

fig.update_layout(
    title='Individual Scoring Trajectories (Green=Improving, Red=Declining, Grey=Stable)',
    xaxis_title='Season Number', yaxis_title='Points Per Game',
    template='plotly_white', height=500
)
fig.show()

The spaghetti plot reveals the messy reality of junior development — there's huge variance. But the average trend line tells a clear story: **players generally improve their scoring output with each additional season.**

The green lines (improving) clearly outnumber the red (declining), which makes sense — kids get bigger, stronger, and more skilled as they age up.

## 🚀 Breakout Players: Biggest PPG Jumps

Which players had the most dramatic improvement between consecutive seasons? These are the breakout stars — the kids who suddenly "got it."

In [8]:
# Calculate season-over-season PPG change
ps_sorted = player_season.sort_values(['player_id', 'season_order'])
ps_sorted['prev_ppg'] = ps_sorted.groupby('player_id')['ppg'].shift(1)
ps_sorted['prev_games'] = ps_sorted.groupby('player_id')['games'].shift(1)
ps_sorted['ppg_jump'] = ps_sorted['ppg'] - ps_sorted['prev_ppg']

# Filter: must have played 5+ games in both seasons
breakouts = ps_sorted[
    (ps_sorted['prev_ppg'].notna()) & 
    (ps_sorted['games'] >= 5) & 
    (ps_sorted['prev_games'] >= 5)
].copy()

top20 = breakouts.nlargest(20, 'ppg_jump')[
    ['first_name', 'last_name', 'season_name', 'age_group', 'prev_ppg', 'ppg', 'ppg_jump', 'games']
].copy()
top20.columns = ['First', 'Last', 'Season', 'Age Group', 'Prev PPG', 'New PPG', 'Jump', 'Games']
top20['Prev PPG'] = top20['Prev PPG'].round(1)
top20['New PPG'] = top20['New PPG'].round(1)
top20['Jump'] = top20['Jump'].round(1)

fig = go.Figure(go.Bar(
    y=[f"{r['First']} {r['Last']}" for _, r in top20.iterrows()],
    x=top20['Jump'],
    orientation='h',
    marker_color=px.colors.sequential.Viridis[::-1][:20],
    text=[f"+{j:.1f} PPG" for j in top20['Jump']],
    textposition='outside',
    hovertext=[f"{r['Prev PPG']} → {r['New PPG']} PPG ({r['Season']}, {r['Age Group']})" for _, r in top20.iterrows()]
))
fig.update_layout(
    title='Top 20 Biggest PPG Jumps Between Consecutive Seasons',
    xaxis_title='PPG Improvement', yaxis=dict(autorange='reversed'),
    template='plotly_white', height=600, margin=dict(l=150)
)
fig.show()

print(top20.to_string(index=False))

          First           Last         Season Age Group  Prev PPG  New PPG  Jump  Games
         Si Jin            Kim Summer 2025/26       U14       8.3     28.0  19.7     11
          Louis            Sun Summer 2025/26       U16       3.2     22.7  19.5     10
         ziqian          kuang Summer 2024/25       U12       4.5     23.9  19.4      9
Ethan Alexander            Joe    Winter 2025       U13      10.1     28.5  18.5     17
         Jeremy          Smith    Winter 2025       U11       1.4     18.8  17.4     15
          Felix             Li Summer 2024/25       U14       1.0     17.3  16.3     15
          ASTON MARSHALL-CHUNG    Winter 2024       U11       0.2     16.4  16.2     15
           Lana      Stagliano Summer 2024/25       U08       2.1     18.2  16.1     15
         Lukesh  Krishnapillai Summer 2024/25       U14       2.4     18.3  15.9      7
          Oscar          McLay Summer 2025/26       U14       6.3     22.2  15.9     11
          Elias         Harika S

## 🎯 Position Proxy: Do Players Change Style as They Age?

Junior basketball doesn't formally track positions, but we can infer play style from scoring patterns. Players who rely heavily on 3-pointers are likely perimeter players (guards), while those scoring mostly on 2-pointers are probably forwards/centres.

Let's see if players shift their scoring style as they move through age groups.

In [9]:
# Need enough shot data to be meaningful
style_df = df[
    (df['games_played'] >= 5) & 
    ((df['two_point'] + df['three_point']) > 0)
].copy()

style_df['total_fg'] = style_df['two_point'] + style_df['three_point']
style_df['pct_3pt'] = style_df['three_point'] / style_df['total_fg']
style_df['pct_2pt'] = style_df['two_point'] / style_df['total_fg']
style_df['ft_rate'] = style_df['one_point'] / style_df['total_points'].clip(lower=1)

# Classify play style
def classify_style(row):
    if row['pct_3pt'] > 0.3:
        return 'Perimeter'
    elif row['pct_3pt'] < 0.1:
        return 'Interior'
    else:
        return 'Balanced'

style_df['style'] = style_df.apply(classify_style, axis=1)

# Age group ordering
ag_order = ['U08', 'U09', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16', 'U17', 'U18', 'U19', 'U20', 'U21', 'U23']
style_df['age_group'] = pd.Categorical(style_df['age_group'], categories=ag_order, ordered=True)
style_df = style_df.dropna(subset=['age_group'])

# Avg 3PT% by age group
ag_style = style_df.groupby('age_group', observed=True).agg(
    avg_3pt_pct=('pct_3pt', 'mean'),
    avg_ft_rate=('ft_rate', 'mean'),
    players=('player_id', 'nunique')
).reset_index()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Average 3PT Shot Share by Age Group', 'Play Style Distribution by Age Group'))

fig.add_trace(go.Scatter(
    x=ag_style['age_group'].astype(str), y=ag_style['avg_3pt_pct'] * 100,
    mode='lines+markers', line=dict(width=3, color='#F44336'),
    marker=dict(size=10), name='3PT %'
), row=1, col=1)

# Style distribution
style_dist = style_df.groupby(['age_group', 'style'], observed=True).size().unstack(fill_value=0)
style_pct = style_dist.div(style_dist.sum(axis=1), axis=0) * 100

for style, color in [('Interior', '#2196F3'), ('Balanced', '#FF9800'), ('Perimeter', '#F44336')]:
    if style in style_pct.columns:
        fig.add_trace(go.Bar(
            x=style_pct.index.astype(str), y=style_pct[style],
            name=style, marker_color=color
        ), row=1, col=2)

fig.update_layout(
    barmode='stack', template='plotly_white', height=450,
    title='How Scoring Style Evolves with Age'
)
fig.update_yaxes(title_text='3PT Shot %', row=1, col=1)
fig.update_yaxes(title_text='% of Players', row=1, col=2)
fig.show()

As expected, younger players (U08–U10) almost never shoot three-pointers — the ball is heavy and the hoop is far away! As players age into U14+, we see a steady shift toward perimeter shooting, mirroring the broader trend in modern basketball.

This isn't just about physical development — it reflects coaching emphasis and the influence of the NBA's three-point revolution filtering down to junior levels.

## 🤖 Can We Predict Future Stars?

Here's the million-dollar question: if we know a player's stats from their first season, can we predict how they'll score in their second season?

We'll build a simple linear regression model to find out.

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error

# Get first and second season stats for each player
first_second = player_season.groupby('player_id').filter(lambda x: len(x) >= 2)
first_season = first_second.groupby('player_id').nth(0).reset_index()
second_season = first_second.groupby('player_id').nth(1).reset_index()

model_df = first_season[['player_id', 'ppg', 'games', 'fg2', 'fg3', 'ft', 'fouls']].merge(
    second_season[['player_id', 'ppg']],
    on='player_id', suffixes=('_s1', '_s2')
)

# Features: first season stats
features = ['ppg_s1', 'games', 'fg2', 'fg3', 'ft', 'fouls']
X = model_df[features].fillna(0)
y = model_df['ppg_s2']

# Cross-validated model
model = LinearRegression()
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
model.fit(X, y)
y_pred = model.predict(X)

print(f"📊 Linear Regression: First Season → Second Season PPG")
print(f"   R² (5-fold CV): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"   R² (train):     {r2_score(y, y_pred):.3f}")
print(f"   MAE:            {mean_absolute_error(y, y_pred):.2f} PPG")

# Feature importance
importance = pd.DataFrame({
    'Feature': features,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=True)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Predicted vs Actual (Season 2 PPG)', 'Feature Importance'))

fig.add_trace(go.Scatter(
    x=y, y=y_pred, mode='markers',
    marker=dict(size=4, opacity=0.3, color='#2196F3'),
    name='Players'
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=[0, y.max()], y=[0, y.max()],
    mode='lines', line=dict(dash='dash', color='red'),
    name='Perfect'
), row=1, col=1)

fig.add_trace(go.Bar(
    y=importance['Feature'], x=importance['Coefficient'],
    orientation='h', marker_color=['#F44336' if c < 0 else '#4CAF50' for c in importance['Coefficient']],
    name='Coefficient'
), row=1, col=2)

fig.update_layout(template='plotly_white', height=450, showlegend=False,
                  title='Predicting Second-Season Scoring from First-Season Stats')
fig.update_xaxes(title_text='Actual PPG', row=1, col=1)
fig.update_yaxes(title_text='Predicted PPG', row=1, col=1)
fig.show()

📊 Linear Regression: First Season → Second Season PPG
   R² (5-fold CV): 0.488 ± 0.015
   R² (train):     0.490
   MAE:            1.76 PPG


The model shows that **first-season PPG is by far the strongest predictor** of second-season scoring. This shouldn't be surprising — the best predictor of future performance is past performance.

But notice the scatter: there's meaningful variance, especially for mid-range scorers. That's where development, coaching, and growth spurts make the difference. The model gives us a baseline, but basketball development isn't deterministic.

## 📐 Age Group Benchmarks: What Does "Good" Look Like?

Parents and coaches constantly ask: "is 6 points per game good for an Under-12?" Here are the percentile benchmarks for key age groups.

In [11]:
# Filter to players with 5+ games for meaningful stats
bench_df = player_season[player_season['games'] >= 5].copy()

age_groups_of_interest = ['U10', 'U12', 'U14', 'U16']
percentiles = [10, 25, 50, 75, 90, 95]

benchmark_data = []
for ag in age_groups_of_interest:
    ag_data = bench_df[bench_df['age_group'] == ag]['ppg']
    if len(ag_data) < 10:
        continue
    row = {'Age Group': ag, 'Players': len(ag_data)}
    for p in percentiles:
        row[f'P{p}'] = round(ag_data.quantile(p/100), 1)
    benchmark_data.append(row)

bench_table = pd.DataFrame(benchmark_data)
print("🏀 Scoring Benchmarks (PPG) by Age Group — Players with 5+ Games")
print("=" * 70)
print(bench_table.to_string(index=False))
print()
print("Reading guide: P50 = median, P75 = top quarter, P90 = elite")

🏀 Scoring Benchmarks (PPG) by Age Group — Players with 5+ Games
Age Group  Players  P10  P25  P50  P75  P90  P95
      U10     7796  0.2  0.8  1.9  3.9  6.4  8.1
      U12     9705  0.5  1.3  2.8  5.0  7.6  9.4
      U14     9238  0.8  1.8  3.4  6.0  8.9 10.9
      U16     7466  0.9  2.0  3.8  6.5  9.5 11.6

Reading guide: P50 = median, P75 = top quarter, P90 = elite


In [12]:
# Visualize as box plots
box_data = bench_df[bench_df['age_group'].isin(['U08','U09','U10','U11','U12','U13','U14','U15','U16'])].copy()
ag_order_filtered = [ag for ag in ag_order if ag in box_data['age_group'].unique()]
box_data['age_group'] = pd.Categorical(box_data['age_group'], categories=ag_order_filtered, ordered=True)

fig = px.box(
    box_data, x='age_group', y='ppg', color='gender',
    color_discrete_map={'Boys': '#2196F3', 'Girls': '#E91E63'},
    title='Points Per Game Distribution by Age Group and Gender',
    labels={'ppg': 'Points Per Game', 'age_group': 'Age Group'}
)
fig.update_layout(template='plotly_white', height=500)
fig.update_yaxes(range=[0, box_data['ppg'].quantile(0.98)])
fig.show()

The benchmarks reveal clear patterns:

- **Scoring increases with age** — median PPG roughly doubles from U10 to U16
- **Variance increases too** — the gap between average and elite widens as players specialize
- **The gender gap emerges around U12-U14** — likely reflecting differences in physical development timing and participation rates

## 🎯 Key Takeaways

1. **Retention is real but imperfect** — About half of all players appear in only one season. For those who stick around, the data shows clear development patterns.

2. **Most multi-season players improve** — The majority of 3+ season players show scoring improvement, validating that consistent participation drives development.

3. **First-season stats predict (but don't determine) the future** — A simple model using first-season stats can explain a meaningful chunk of second-season performance, but there's plenty of room for growth, coaching impact, and late bloomers.

4. **Scoring style evolves with age** — Young players are almost exclusively interior scorers. By U14-U16, three-point shooting becomes a significant part of the game, reflecting both physical maturation and modern basketball trends.

5. **Benchmarks exist** — We now have data-driven percentile tables for what "good" scoring looks like at each age group — far better than the vibes-based assessments that currently dominate junior basketball evaluation.

---

*This analysis is part of the [FullCourtVision](https://github.com/yourusername/FullCourtVision) project — data-driven insights into Victorian junior basketball.*